In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
import pickle
from flowers.utils.customDataset import CustomDataset

data = pickle.load(open("./data/Cifar10/train/0.pkl", "rb"))


ModuleNotFoundError: No module named 'utils'

In [52]:
# Create a DataLoader
batch_size = 32  # Adjust this as needed
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

In [53]:
dataloader.dataset[0]["data"].shape

(32, 32, 3)


TypeError: Cannot handle this data type: (1, 1, 3), <f4

In [37]:
import numpy as np
import os
import sys
import random
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, TensorDataset, DataLoader, Dataset
from PIL import Image
import pickle

from flowers.non_iid_generator.utils.dataset_utils import check, separate_data, split_data, save_file

from flowers.utils.customDataset import CustomDataset
from argparse import ArgumentParser
dir_path = "./data/Cifar10/"

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        np_img = np.array(img, dtype = np.uint8)
        # print(type(np_img))
        # print(np_img.shape)
        img = Image.fromarray(np_img, "RGB")

        if self.transform is not None:
            img = self.transform(img)
            # print("-->", type(img))
            # print(type(img))
            # print(img.shape)

        class_id = torch.tensor([self.labels[idx]])

        sample = {
            'data': img,
            'label': self.labels[idx]
        }
        
        return img, class_id

In [38]:
# transform = transforms.Compose(
#     [
#         transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#     ]
# )

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4), 
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(
    root=dir_path+"rawdata", train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(
    root=dir_path+"rawdata", train=False, download=True, transform=transform)

# ttset = ConcatDataset([trainset, trainset])

# ttloader = torch.utils.data.DataLoader(ttset, batch_size=len(trainset.data), shuffle=False)

# trainloader = torch.utils.data.DataLoader(
#     trainset, batch_size=len(trainset.data), shuffle=False)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=len(testset.data), shuffle=False)

dataset_image = []
dataset_image.extend(trainset.data)
dataset_image.extend(testset.data)
dataset_image = np.array(dataset_image)

train_targets = trainset.targets
test_targets = testset.targets

dataset_label = []
dataset_label.extend(train_targets)
dataset_label.extend(test_targets)
dataset_label = np.array(dataset_label)



Files already downloaded and verified
Files already downloaded and verified


In [39]:
alpha = 0.1
batch_size = 32
def save_file(config_path, train_path, test_path, train_data, test_data, num_clients, 
                num_classes, statistic, niid=False, balance=True, partition=None):
    config = {
        'num_clients': num_clients, 
        'num_classes': num_classes, 
        'non_iid': niid, 
        'balance': balance, 
        'partition': partition, 
        'Size of samples for labels in clients': statistic, 
        'alpha': alpha, 
        'batch_size': batch_size, 
    }

    # gc.collect()
    print("Saving to disk.\n")

    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4), 
        transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    for idx, train_dict in enumerate(train_data):
        data = train_dict["x"]
        labels = train_dict["y"]
        # tensor_x = torch.Tensor(data) # transform to torch tensor
        # tensor_y = torch.Tensor(labels)

        # my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
        # my_dataloader = DataLoader(my_dataset) # create your dataloader


        # for idx, train_data in enumerate(my_dataloader, 0):
        #     data, labels = train_data
        #     print(data)


        
        data = train_dict["x"]
        labels = train_dict["y"]
        custom_dataset = CustomDataset(data, labels, transform=transform)
        print(custom_dataset[0])

        with open(train_path + str(idx) + '.pkl', 'wb') as f:
            pickle.dump(custom_dataset, f)

    # for idx, test_dict in enumerate(test_data):
    #     with open(test_path + str(idx) + '.npz', 'wb') as f:
    #         np.savez_compressed(f, data=test_dict)
    # with open(config_path, 'w') as f:
    #     ujson.dump(config, f)

    # print("Finish generating dataset.\n")

In [40]:
num_clients = 3
num_classes = 10
niid = True
balance = False
partition = "dir"

X, y, statistic = separate_data((dataset_image, dataset_label), num_clients, num_classes, 
                                niid, balance, partition)

train_data, test_data = split_data(X, y)

Client 0	 Size of data: 23214	 Labels:  [0 3 4 6 7 8]
		 Samples of labels:  [(0, 5240), (3, 1303), (4, 5994), (6, 4934), (7, 321), (8, 5422)]
--------------------------------------------------
Client 1	 Size of data: 16854	 Labels:  [0 1 2 3 4 5 6 7 9]
		 Samples of labels:  [(0, 31), (1, 258), (2, 5999), (3, 1009), (4, 5), (5, 5999), (6, 1060), (7, 387), (9, 2106)]
--------------------------------------------------
Client 2	 Size of data: 19932	 Labels:  [0 1 2 3 4 5 6 7 8 9]
		 Samples of labels:  [(0, 729), (1, 5742), (2, 1), (3, 3688), (4, 1), (5, 1), (6, 6), (7, 5292), (8, 578), (9, 3894)]
--------------------------------------------------
Total number of samples: 60000
The number of train samples: [17410, 12640, 14949]
The number of test samples: [5804, 4214, 4983]



In [41]:
config_path = dir_path + "config.json"
train_path = dir_path + "train/"
test_path = dir_path + "test/"

save_file(config_path, train_path, test_path, train_data, test_data, num_clients, num_classes, 
        statistic, niid, balance, partition)

Saving to disk.

(tensor([[[-0.8201, -0.8201, -0.8201,  ..., -2.4291, -2.4291, -2.4291],
         [-0.8201, -0.8201, -0.8201,  ..., -2.4291, -2.4291, -2.4291],
         [-0.8201, -0.8201, -0.8201,  ..., -2.4291, -2.4291, -2.4291],
         ...,
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
         [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291]],

        [[-0.3532, -0.3532, -0.3532,  ..., -2.4183, -2.4183, -2.4183],
         [-0.3532, -0.3532, -0.3532,  ..., -2.4183, -2.4183, -2.4183],
         [-0.3532, -0.3532, -0.3532,  ..., -2.4183, -2.4183, -2.4183],
         ...,
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
         [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183]],

        [[-1.1483, -1.1483, -1.1483,  ..., -2.2214, -2.2214, -2.2214],
         [-1.1483, -1.1483,

In [49]:
data = pickle.load(open("./data/Cifar10/train/0.pkl", "rb"))
batch_size = 32  # Adjust this as needed
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

ModuleNotFoundError: No module named 'utils'

17410

In [73]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = torch.load("models/alexnet/model_pt21.pth.tar")

criterion = torch.nn.BCEWithLogitsLoss()
correct, loss = 0, 0.0
with torch.no_grad():
    for images, labels in dataloader:
        
        # labels.unsqueeze_(1)
        target_onehot = torch.FloatTensor(labels.shape[0], 10)
        target_onehot.zero_()
        target_onehot.scatter_(1, labels, 1)
        # labels.squeeze_(1)
        labels = labels.to(DEVICE)

        outputs = net(images.to(DEVICE))
        labels_one_hot = target_onehot.to(DEVICE)

        loss += criterion(outputs, labels_one_hot).item()
        correct += (torch.max(outputs.data, 1)[1] == labels.squeeze_(1)).sum().item()
        # print(torch.max(outputs.data, 1)[1].shape)
        # print(labels.squeeze(1).shape)
        # print((torch.max(outputs.data, 1)[1] == labels.squeeze(1)).shape)
        print(correct)
        print(f"ACC: {correct / len(dataloader.dataset)}")

31
ACC: 0.0017805858701895462
63
ACC: 0.0036186099942561744
95
ACC: 0.005456634118322803
127
ACC: 0.007294658242389431
159
ACC: 0.00913268236645606
189
ACC: 0.010855829982768524
220
ACC: 0.01263641585295807
252
ACC: 0.014474439977024698
284
ACC: 0.01631246410109133
315
ACC: 0.01809304997128087
347
ACC: 0.0199310740953475
379
ACC: 0.02176909821941413
410
ACC: 0.023549684089603676
442
ACC: 0.025387708213670306
474
ACC: 0.02722573233773693
505
ACC: 0.029006318207926478
537
ACC: 0.030844342331993107
569
ACC: 0.032682366456059736
600
ACC: 0.03446295232624928
630
ACC: 0.03618609994256174
662
ACC: 0.038024124066628376
694
ACC: 0.039862148190695
726
ACC: 0.041700172314761634
758
ACC: 0.04353819643882826
788
ACC: 0.04526134405514072
819
ACC: 0.04704192992533027
850
ACC: 0.048822515795519814
882
ACC: 0.050660539919586446
914
ACC: 0.05249856404365307
946
ACC: 0.054336588167719704
978
ACC: 0.05617461229178633
1009
ACC: 0.05795519816197588
1041
ACC: 0.0597932222860425
1072
ACC: 0.06157380815623205


KeyboardInterrupt: 